In [1]:
from urllib.request import urlopen
from io import BytesIO
from PIL import Image
from imagehash import phash
from pyspark.sql import functions as F
from pyspark.sql import types as T
import numpy as np

In [2]:
df = sqlContext.read.parquet("/guoda/data/idigbio-media-20170709T013207-100k.parquet")

In [10]:
df2 = df.sample(False,0.005,45) # to select small dataframe for faster execution 
df2.count()

513

In [3]:
def imagebytes(v):
    try:
        b = BytesIO()
        sqr = (Image.open(BytesIO(urlopen("http://api.idigbio.org/v2/media?size=thumbnail&filereference=" + v).read()))).resize((256,256),Image.LANCZOS)
        sqr.save(b,format="jpeg")
        b.seek(0)
        return b.read()
    except:
        return np.nan

In [6]:
imgb_udf = F.udf(imagebytes, T.BinaryType())
newdf = df2.select("accessuri",imgb_udf("accessuri"))

In [7]:
newdf.printSchema()
newdf.groupby("imagebytes(accessuri)").count().sort(F.col("count").desc()).show()

root
 |-- accessuri: string (nullable = true)
 |-- imagebytes(accessuri): binary (nullable = true)

+---------------------+-----+
|imagebytes(accessuri)|count|
+---------------------+-----+
|                 null|  134|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
| [FF D8 FF E0 00 1...|    1|
+---------------------+-----+
only showing top 20 rows



In [8]:
newdf.show()

+--------------------+---------------------+
|           accessuri|imagebytes(accessuri)|
+--------------------+---------------------+
|http://mediaphoto...| [FF D8 FF E0 00 1...|
|http://mediaphoto...| [FF D8 FF E0 00 1...|
|http://mediaphoto...| [FF D8 FF E0 00 1...|
|http://mediaphoto...| [FF D8 FF E0 00 1...|
|http://bovary.ipl...|                 null|
|http://bovary.ipl...|                 null|
|http://bovary.ipl...|                 null|
|http://arctos.dat...| [FF D8 FF E0 00 1...|
|http://mediaphoto...| [FF D8 FF E0 00 1...|
|http://mediaphoto...| [FF D8 FF E0 00 1...|
|http://mediaphoto...| [FF D8 FF E0 00 1...|
|http://cdm15705.c...| [FF D8 FF E0 00 1...|
|http://mediaphoto...| [FF D8 FF E0 00 1...|
|http://mediaphoto...| [FF D8 FF E0 00 1...|
|http://mediaphoto...| [FF D8 FF E0 00 1...|
|http://mediaphoto...| [FF D8 FF E0 00 1...|
|http://mediaphoto...| [FF D8 FF E0 00 1...|
|http://mediaphoto...| [FF D8 FF E0 00 1...|
|http://mediaphoto...| [FF D8 FF E0 00 1...|
|http://me